# Drifting y Quality

Un desafío común para los **data scientists** en el mundo real es que, a diferencia de las competencia de **Kaggle**, los datos cambian con el tiempo. Esto significa que los datos que recibiremos en el futuro no necesariamente reflejarán los que usamos para entrenar nuestros modelos. Esta homogeneidad es esencial, ya que los algoritmos aprenden patrones en función de las distribuciones observadas durante el entrenamiento. Si esas distribuciones cambian, la capacidad predictiva del modelo se deteriora.

Podemos identificar dos tipos principales de problemáticas asociadas a estos cambios:

- **Data Quality (DQ)**
- **Data Drift (DF)**

Comencemos con el primero: **Data Quality (DQ)**. Hay mucho que decir sobre este tema, pero mencionaremos brevemente que los datos provienen de procesos creados por seres humanos, quienes son propensos a cometer errores y hacer modificaciones. A lo largo del ciclo de vida de los modelos, es común que se enfrenten a este tipo de situaciones. Por ello, es fundamental implementar controles que garanticen que los datos a los que se aplican los modelos mantienen la misma estructura que los datos usados en el entrenamiento.

A continuación, veremos dos indicadores clave para monitorear la calidad de los datos, aunque no son los únicos a considerar.


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from os import path

In [2]:
base_path = '/content/drive/MyDrive/DMEyF/2025/'
dataset_path = base_path + 'datos/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01.csv'

ganancia_acierto = 780000
costo_estimulo = 20000

data = pd.read_csv(path.join(dataset_path,dataset_file))

Trabajaremos con un mes de entrenamiento, pero pueden ser muchos más y el mes donde finalmente haremos la predicción.

In [4]:
mes_train = 202102
mes_score = 202104


El primer control es la cantidad de **valores nulos** por variable. Cuando ocurre un error en el proceso de generación de datos, es común que se manifieste un aumento en la cantidad de valores nulos.


In [5]:
train_data = data[data['foto_mes'] == mes_train]
score_data = data[data['foto_mes'] == mes_score]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted

,Train Null Percentage,Score Null Percentage,diff
mtarjeta_master_descuentos,0.000000,3.306857,3.306857
mtarjeta_visa_descuentos,0.000000,1.691368,1.691368
Visa_Finiciomora,99.384539,99.123720,0.260820
Master_mpagospesos,59.820542,59.584012,0.236531
Master_cadelantosefectivo,59.820542,59.584012,0.236531
...,...,...,...
ccajas_otras,0.000000,0.000000,0.000000
catm_trx,0.000000,0.000000,0.000000
matm,0.000000,0.000000,0.000000
catm_trx_other,0.000000,0.000000,0.000000


Un problema similar es la sobrerrepresentación de ceros en los datos, ya que este valor se utiliza comúnmente para la imputación. La acumulación excesiva de ceros puede ser indicativa de un error o de un proceso de imputación inadecuado.

In [6]:
train_zero_percentage = (train_data == 0).mean() * 100
score_zero_percentage = (score_data == 0).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Zero Percentage': train_zero_percentage, 'Score Zero Percentage': score_zero_percentage})

comparison_df_zero['diff_zero_percentage'] = (comparison_df_zero['Score Zero Percentage'] - comparison_df_zero['Train Zero Percentage']).abs()
diff_zero_percentage_sorted = comparison_df_zero.sort_values('diff_zero_percentage',ascending=False)
diff_zero_percentage_sorted


,Train Zero Percentage,Score Zero Percentage,diff_zero_percentage
ctarjeta_visa_descuentos,100.000000,94.335997,5.664003
mtarjeta_visa_descuentos,100.000000,94.573425,5.426575
mcuenta_corriente,52.476951,47.192476,5.284475
mcomisiones_mantenimiento,71.088773,66.160398,4.928375
ctarjeta_master_descuentos,100.000000,95.346902,4.653098
...,...,...,...
Master_fechaalta,0.000000,0.000000,0.000000
Visa_fultimo_cierre,0.000000,0.000000,0.000000
Visa_mlimitecompra,0.000000,0.000000,0.000000
Visa_fechaalta,0.000000,0.000000,0.000000


La segunda problemática es el **data drifting**, que es un fenómeno que ocurre cuando la distribución de los datos cambia con el tiempo.

Hay varios tipos de drifting:

1. **Feature Drift**: Este tipo de deriva se da cuando cambia la distribución de una feature del modelo.

2. **Concept Drift**: Ocurre cuando cambia la relación entre las features y la variable target. Por ejemplo, en un modelo de predicción de fraude, la forma en que los fraudes ocurren puede cambiar con el tiempo, lo que significa que el modelo necesitaría ser ajustado para reconocer nuevos patrones.

Para detectar el drifting utilizaremos el **PSI (Population Stability Index)** que es una métrica utilizada para medir los cambios en la distribución de una variable a lo largo del tiempo. Funciona cuantificando las diferencias entre dos distribuciones, generalmente comparando los datos de entrenamiento con los datos actuales.

* **Cómo funciona el PSI**:
 1. **División en intervalos (bins)**: Tanto los datos históricos como los datos actuales se dividen en una serie de intervalos o bins.
   
 2. **Cálculo de frecuencias**: Se calcula la proporción de observaciones que caen en cada bin tanto para la distribución original (entrenamiento) como para la nueva (actual).

 3. **Fórmula del PSI**: Para cada bin, se calcula la diferencia entre las frecuencias observadas en ambas distribuciones mediante la siguiente fórmula:

   $
   PSI = \sum \left( (P_{actual} - P_{esperado}) \times \log \left( \frac{P_{actual}}{P_{esperado}} \right) \right)
   $
   Donde:
   - $P_{actual}$ es la proporción de la nueva muestra en un bin.
   - $P_{esperado}$ es la proporción de la muestra original en el mismo bin.



In [7]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null


In [8]:
buckets=10
column = 'mprestamos_personales'
expected = train_data[column]
actual = score_data[column]

def psi_formula(expected_prop, actual_prop):
    result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
    return result

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

expected_prop = expected_counts / len(expected_not_null)
actual_prop = actual_counts / len(actual_not_null)

psi_not_null = psi_formula(expected_prop, actual_prop).sum()

psi_null = 0

psi_value = psi(expected, actual)


In [9]:
expected_counts

array([129724,  16215,  16216])

In [10]:
actual_counts

array([130052,  15819,  17545])

In [17]:
psi_value

np.float64(0.004551929338053304)

Y aplicamos el análisis a (casi) todos los **features**

In [11]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria', 'ctarjeta_master_descuentos','mtarjeta_master_descuentos','mtarjeta_visa_descuentos','ctarjeta_visa_descuentos','ccajeros_propios_descuentos','mcajeros_propios_descuentos']:
    print(column)
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)
psi_df


numero_de_cliente
active_quarter
cliente_vip
internet
cliente_edad
cliente_antiguedad
mrentabilidad
mrentabilidad_annual
mcomisiones
mactivos_margen
mpasivos_margen
cproductos
tcuentas
ccuenta_corriente
mcuenta_corriente_adicional
mcuenta_corriente
ccaja_ahorro
mcaja_ahorro
mcaja_ahorro_adicional
mcaja_ahorro_dolares
cdescubierto_preacordado
mcuentas_saldo
ctarjeta_debito
ctarjeta_debito_transacciones
mautoservicio
ctarjeta_visa
ctarjeta_visa_transacciones
mtarjeta_visa_consumo
ctarjeta_master
ctarjeta_master_transacciones
mtarjeta_master_consumo
cprestamos_personales
mprestamos_personales
cprestamos_prendarios
mprestamos_prendarios
cprestamos_hipotecarios
mprestamos_hipotecarios
cplazo_fijo
mplazo_fijo_dolares
mplazo_fijo_pesos
cinversion1
minversion1_pesos
minversion1_dolares
cinversion2
minversion2
cseguro_vida
cseguro_auto
cseguro_vivienda
cseguro_accidentes_personales
ccaja_seguridad
cpayroll_trx
mpayroll
mpayroll2
cpayroll2_trx
ccuenta_debitos_automaticos
mcuenta_debitos_automati

,feature,psi
129,Visa_Finiciomora,2.830009
107,Master_Finiciomora,2.066463
67,mcomisiones_otras,0.070516
8,mcomisiones,0.069917
106,Master_Fvencimiento,0.069454
...,...,...
91,ccajas_transacciones,0.000000
94,ccajas_extracciones,0.000000
93,ccajas_depositos,0.000000
92,ccajas_consultas,0.000000


Encontramos un par de variables conflictivas **Master_Finiciomora** y **Visa_Finiciomora** que nos dan que hay grandes cambios. Vamos a ver que sucedio, haciendo un poco de **deep dive**.

In [12]:
variable_name = 'Master_Finiciomora'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=3, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')


Cortes en Master_Finiciomora: [np.float64(-1.001), np.float64(20.0), np.float64(146.0)]
Frecuencia Esperada: [ 94 556]
Frecuencia Actual: [606 341]


* Qué paso? y cómo podría arreglarse?

El resto de las variables que vemos son casos ya detectados. Observamos también que hay un cambio en las variables de **payroll**


In [13]:
variable_name = 'cpayroll_trx'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=20, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')

Cortes en cpayroll_trx: [np.float64(-0.001), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(60.0)]
Frecuencia Esperada: [75945 51405 23796 11009]
Frecuencia Actual: [75322 53995 23056 11039]


Pero antes de tomar una decisión, vamos a analizar como los **lgbm** cortan esa variable, para esto importamos nuestro modelo y analizamos sus puntos de corte por cada variable

In [14]:
model = lgb.Booster(model_file='/content/drive/MyDrive/DMEyF/2025/modelos/lgb_first.txt')


In [15]:
model_df = model.trees_to_dataframe()

In [20]:
model_df[model_df['split_feature'] == 'mprestamos_personales'].sort_values('threshold')

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
75,0,5,0-S32,0-S37,0-L33,0-S15,mprestamos_personales,23.451700,1.000000e-35,<=,left,None,-4.479670,264.9400,23328
68,0,5,0-S24,0-S27,0-L25,0-S15,mprestamos_personales,46.495998,1.000000e-35,<=,left,None,-4.399050,45.1335,3974
507,5,4,5-S35,5-L35,5-L36,5-S34,mprestamos_personales,13.100800,1.000000e-35,<=,left,None,-0.021773,33.1631,2914
1261,14,4,14-S20,14-S21,14-L21,14-S11,mprestamos_personales,8.167380,1.000000e-35,<=,left,None,-0.023419,128.0320,14975
1760,20,5,20-S9,20-S16,20-L10,20-S5,mprestamos_personales,19.350599,1.000000e-35,<=,left,None,-0.014333,217.2870,29189
3809,44,3,44-S6,44-S16,44-S22,44-S3,mprestamos_personales,7.093080,1.000000e-35,<=,left,None,-0.008853,209.9260,37406
3879,45,2,45-S3,45-S6,45-L4,45-S0,mprestamos_personales,13.399900,1.000000e-35,<=,left,None,-0.020050,302.9510,92546
3834,45,7,45-S20,45-S21,45-L21,45-S7,mprestamos_personales,5.865810,1.000000e-35,<=,left,None,0.005872,811.5360,25030
4086,48,7,48-S26,48-S32,48-S27,48-S7,mprestamos_personales,5.667850,7.448335e+03,<=,left,None,0.007215,1024.0700,42869
1237,14,5,14-S13,14-S14,14-L14,14-S10,mprestamos_personales,10.897100,7.448335e+03,<=,left,None,-0.018666,338.9360,37619


* En que valores corta la variable **cpayroll_trx**?
* Cómo puede corregir el drifting si este está presente en una **feature** importante?